In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
SWING_TRADE_DURATION = 5
SOURCE_FILE = "./input/PETR4_Daily"

In [ ]:
StockPrices = pd.read_csv("{}.csv".format(SOURCE_FILE))
StockPrices['Counter'] = np.arange(len(StockPrices))

In [ ]:
def get_previous_stock_price(df, x):
    if x['Counter'] > (SWING_TRADE_DURATION - 1):
        return float(df[df['Counter'] == (x['Counter'] - SWING_TRADE_DURATION)]['Price'])
    else:
        return 0

def calculate_current_trend(x):
    if x['PreviousStartPrice'] > 0:
        return ((float(x['Price']) / float(x['PreviousStartPrice'])) - 1) * 100
    else:
        return 0;

def get_future_stock_price(df, x, max):
    if x['Counter'] < (max - SWING_TRADE_DURATION):
        return float(df[df['Counter'] == (x['Counter'] + SWING_TRADE_DURATION)]['Price'])
    else:
        return 0

def calculate_future_trend(x):
    if x['FutureFinalPrice'] > 0:
        return ((float(x['Price']) / float(x['FutureFinalPrice'])) - 1) * 100
    else:
        return 0;

def calculate_swing_strenght(x):
    if float(x['CurrentTrend']) != 0 and float(x['FutureTrend'] != 0):
        return abs(float(x['CurrentTrend']) - float(x['FutureTrend']))
    else:
        return 0
    
def correct_date(x):
    date_str = str(x['Date'])
    return date_str[0:4] + '-' + date_str[4:6] + '-' + date_str[6:8]    

In [ ]:
max_counter = StockPrices['Counter'].max()

StockPrices['CorrectedDate'] = StockPrices.apply( lambda x :  correct_date(x), axis =1 )
StockPrices['PreviousStartPrice'] = StockPrices.apply( lambda x :  get_previous_stock_price(StockPrices, x), axis =1 )
StockPrices['CurrentTrend'] = StockPrices.apply(lambda x : calculate_current_trend(x), axis = 1)

StockPrices['FutureFinalPrice'] = StockPrices.apply( lambda x :  get_future_stock_price(StockPrices, x, max_counter), axis =1 )
StockPrices['FutureTrend'] = StockPrices.apply(lambda x : calculate_future_trend(x), axis = 1)
StockPrices['SwingStrength'] = StockPrices.apply(lambda x: calculate_swing_strenght(x), axis =1)


In [ ]:
#StockPrices[StockPrices['SwingStrength'] > 13]

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=StockPrices['CorrectedDate'],
                open=StockPrices['Open'],
                high=StockPrices['High'],
                low=StockPrices['Low'],
                close=StockPrices['Price'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
output_excel_file='./output/PETR4.Analisys.xlsx'
StockPrices.to_excel(output_excel_file)